In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import figure, animation
import scipy as sci
mpl.rcParams.update({
    'font.family': 'serif',
    'text.usetex': True,
    'axes.labelsize': 'large',
    'figure.dpi': 100,
})

import numba
import os
import glob
import subprocess
import re

In [2]:
def fig_io(fig, savefig, viewfig):
    '''
    Save figure and optionaly open it in an image viewer.
    '''
    if (not savefig) and viewfig:
        raise ValueError(
            'Setting `viewfig=True` is only'
            'valid when `savefig` is not `None`.'
        )
        
    if savefig:
        lowername = str.lower(savefig)
        is_img = (
            str.endswith(lowername, '.png')
            or str.endswith(lowername, '.jpg')
            or str.endswith(lowername, '.webp')
        )
        is_pdf = str.endswith(lowername, '.pdf')
        
        if not (is_img or is_pdf):
            raise ValueError(
                'Name for saved file `savefig` must have'
                'extension pdf, png, jpg or webp, '
                f'`{savefig}` is not a valid `savefig` value.'
            )
        
        # Save figure.
        absname = os.path.abspath(savefig)
        fig.savefig(absname, dpi=300)
            
        if viewfig:
            cmd = 'qview' if is_img else 'zathura'
            subprocess.Popen([cmd, absname])

In [3]:
# Import all measurements from the `./measurements` directory. Each measurement file
# is read into a numpy array (column-first) and added to the `meas_dict` (keys are
# filenames with extentions removed).
meas_dict = {}
suffix, sep = 'csv', ','
for fname in glob.glob(f'./measurements/*.{suffix}'):
    with open(fname) as file:
        meas = [[float(el) for el in row.rstrip().split(sep) if el]
                for row in file.readlines() if not (str.startswith(row, '#') or str.isspace(row))]
        meas_name, _ = str.rsplit(os.path.basename(fname), '.', maxsplit=1)
        meas_dict[meas_name] = np.array(meas).T

In [89]:
def fig_ionization_lost(
    meas_name,
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([6, 7])
    axs = fig.subplots(2, 1)
    ax1, ax2 = axs
    
    i, A, x = meas_dict['bolometer-b']
    t = i * 1e-4 - 0.3
    
    ax1.plot(
        1e3 * t, -A,
        label=r'sonda'
    )
    ax1b = ax1.twinx()
    ax1b.plot(
        1e3 * t, (x - np.min(x))/(np.max(x) - np.min(x)),
        color='tab:orange',
        label=r'potenciometer'
    )
    ax1b.set(ylabel='potenciometer $[0, 1]$')
    a, b = -20e-3, 208e-3
    ax1.fill_between(
        [1e3 * a, 1e3 * b], [100, 100], -100,
        color='darkslategrey', alpha=0.2
    )
    
    i, A, x = meas_dict['hardstop-b']
    t = i * 1e-4 - 0.3
    
    ax2.plot(
        1e3 * t, -A,
        label=r'sonda'
    )
    ax2b = ax2.twinx()
    ax2b.plot(
        1e3 * t, (x - np.min(x))/(np.max(x) - np.min(x)),
        color='tab:orange',
        label=r'potenciometer'
    )
    ax2b.set(ylabel='potenciometer $[0, 1]$')
    a, b = -60e-3, 180e-3
    ax2.fill_between(
        [1e3 * a, 1e3 * b], [100, 100], -100,
        color='darkslategrey', alpha=0.2
    )
    
    # --- Set ---
    for ax in axs:
        ax.set(
            xlabel=r'$t\,[\mathrm{ms}]$',
            ylabel=r'signal $[\mathrm{V}]$',
            xlim=(1e3 * -0.3, 1e3 * (6e3*1e-4 - 0.3)),
            ylim=(0.0, 0.7),
            #yscale='log',
        )
        #ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        fig.subplots_adjust(hspace=0.3)
        ax.legend(frameon=False, loc='upper left')
    ax2.set_ylim(-0.5, 5)
    for ax in [ax1b, ax2b]:
        ax.set(xlabel=r'$t\,[\mathrm{ms}]$')
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
    
fig_ionization_lost('', savefig='traces.pdf', viewfig=False)

In [133]:
l = 6.0e-2 - 1.6e-2
Δl = 2e-3

In [176]:
def fig_ionization_lost(
    meas_name,
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([6, 3.8])
    ax = fig.subplots(1, 1)
    ax2 = ax.twinx()
    
    # --- Bolometer ---
    
    i1, A1_, x1 = meas_dict['bolometer-b']
    A1 = -A1_
    t1 = i1 * 1e-4 - 0.3
    x1_01 = (x1 - np.min(x1))/(np.max(x1) - np.min(x1))
    
    def parabola(x, A, a, b):
        return A * (x-a)**2 + b
    
    peak_1a = (x1_01 < 0.3) * (A1 > 0.47)
    #ax.plot(x1_01[peak_1a], A1[peak_1a], color='r', zorder=10)
    par1, cov1 = sci.optimize.curve_fit(parabola, x1_01[peak_1a], A1[peak_1a], p0=[-10, 0.15, 0.6])
    ax.plot(x1_01[peak_1a], parabola(x1_01[peak_1a], *par1), color='0', ls='--', zorder=10)
    _, _, h_1a = par1
    _, _, Δh_1a = np.sqrt(np.diag(cov1))
    
    peak_1b = (x1_01 < 0.5) * (A1 < 0.22)
    #ax.plot(x1_01[peak_1b], A1[peak_1b], color='r', zorder=10)
    par2, cov2 = sci.optimize.curve_fit(parabola, x1_01[peak_1b], A1[peak_1b], p0=[10, 0.35, 0.1])
    ax.plot(x1_01[peak_1b], parabola(x1_01[peak_1b], *par2), color='0', ls='--', zorder=10)
    _, x_1b, h_1b = par2
    _, Δx_1b, Δh_1b = np.sqrt(np.diag(cov2))
    
    # Ubranost
    s = np.sqrt(h_1b/h_1a)
    Δs = np.sqrt(
        (1/(2*np.sqrt(h_1b*h_1a)) * Δh_1b)**2
      + (1/2 * np.sqrt(h_1b/h_1a**3) * Δh_1a)**2
    )
    print(f'h_min = {h_1b:.3f} +/- {Δh_1b:.3f} V')
    print(f'h_max = {h_1a:.3f} +/- {Δh_1a:.3f} V')
    print(f's = {s:.3f} +/- {Δs:.3f}')
    print()
    
    # --- Hard stop ---
    
    i2, A2_, x2 = meas_dict['hardstop-b']
    A2 = -A2_
    t2 = i2 * 1e-4 - 0.3
    x2_01 = (x2 - np.min(x2))/(np.max(x2) - np.min(x2))
    
    peak_2a = (x2_01 < 0.3) * (A2 < 1)
    #ax2.plot(x2_01[peak_2a], A2[peak_2a], color='r', zorder=10)
    par, cov = sci.optimize.curve_fit(parabola, x2_01[peak_2a], A2[peak_2a], p0=[-10, 0.15, 0.6])
    ax2.plot(x2_01[peak_2a], parabola(x2_01[peak_2a], *par), color='0', ls='--', zorder=10)
    _, x_2a, h_2a = par
    _, Δx_2a, Δh_2a = np.sqrt(np.diag(cov))
    
    peak_2b = (x2_01 > 0.4) * (x2_01 < 0.8) * (A2 < 1.2)
    #ax2.plot(x2_01[peak_2b], A2[peak_2b], color='r', zorder=10)
    par, cov = sci.optimize.curve_fit(parabola, x2_01[peak_2b], A2[peak_2b], p0=[10, 0.35, 0.1])
    ax2.plot(x2_01[peak_2b], parabola(x2_01[peak_2b], *par), color='0', ls='--', zorder=10)
    _, x_2b, h_2b = par
    _, Δx_2b, Δh_2b = np.sqrt(np.diag(cov))
    
    λ_01 = 2*(x_2b - x_2a)
    Δλ_01 = 2*(Δx_2a + Δx_2b)
    λ = l*λ_01
    Δλ = np.sqrt( (l*Δλ_01)**2 + (λ_01*Δl)**2 )
    print(f'λ_01 = {λ_01:.4f} +/- {Δλ_01:.4f}')
    print(f'λ = {1e2 * λ:.1f} +/- {1e2 * Δλ:.1f} cm')
    print()
    
    x_min_01 = x_1b - x_2a
    Δx_min_01 = Δx_1b + Δx_2a
    x_min = l * x_min_01
    Δx_min = np.sqrt( (l*Δx_min_01)**2 + (x_min_01*Δl)**2 )
    print(f'x_min_01 = {x_min_01:.4f} +/- {Δx_min_01:.4f}')
    print(f'x_min = {1e2 * x_min:.2f} +/- {1e2 * Δx_min:.2f} cm')
    print()
    
    βx_min = 2*np.pi * x_min_01/λ_01
    Δβx_min = 2*np.pi * np.sqrt( (Δx_min_01/λ_01)**2 + (Δλ_01*x_min_01/λ_01**2) )
    print(f'βx_min = {βx_min:.2f} +/- {Δβx_min:.2f}')
    print()
    
    ηR_rel = (s**2 - 1) * np.tan(βx_min) / ( 1 + s**2 * np.tan(βx_min)**2 )
    #ηR_rel = (s**2 - 1) * np.tan(βx_min) / ( 1 + s**2 * np.tan(βx_min)**2 )
    ΔηR_rel = 0.02
    print(f'ηR_rel = {ηR_rel:.2f} +/- {ΔηR_rel:.2f}')
    print()
    
    ξR_rel = (1 - ηR_rel * np.tan(βx_min)) * s
    ΔξR_rel = 0.04
    print(f'ξR_rel = {ξR_rel:.2f} +/- {ΔξR_rel:.2f}')
    print()

    ZR_rel = np.sqrt(ηR_rel**2 + ξR_rel**2)
    ΔZR_rel = np.sqrt( (ΔηR_rel * ηR_rel/ZR_rel)**2 + (ΔξR_rel * ξR_rel/ZR_rel)**2 )
    print(f'ZR_rel = {ZR_rel:.2f} +/- {ΔZR_rel:.2f}')
    print()
    
    print(f'P = ')
    for P_m_ in [0.1, 0.22, 0.38, 0.56, 0.52, 0.38]:
        P_m = 1e-3 * P_m_
        rR2 = ((1-s) / (1+s))**2
        P = P_m / (1 - rR2)
        ΔP = P_m * ( -1/4 + 1/(4*s**2) ) * Δs
        #print(f'{1e3 * P:.2f} \\pm {1e3 * ΔP:.3f}')
        print(f'{1e3 * P:.2f} mW')
    
    ax.plot(
        x1_01, A1,
        color='tab:green', ls='', marker='.', alpha=0.5,
        label=r'z bolometrom'
    )
    ax2.plot(
        x2_01, A2,
        color='tab:orange', ls='', marker='.', alpha=0.5,
        label=r'zaprt konec'
    )
    
    # --- Set ---
    ax.set(
        xlabel=r'potenciometer $[0, 1]$',
        ylabel=r'z bolometrom $[\mathrm{V}]$',
        xlim=(0, 1),
        ylim=(0,0.7),
        #yscale='log',
    )
    ax2.set(
        ylabel=r'zaprt konec $[\mathrm{V}]$',
        xlim=(0, 1),
        ylim=(0, 4.8)
    )
    #ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    fig.subplots_adjust(bottom=0.16, top=0.85, hspace=0.3)
    fig.legend(frameon=False, loc='upper center', bbox_to_anchor=[0.5, 0.96], ncols=2)
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
    
fig_ionization_lost('', savefig='s.pdf', viewfig=False)

h_min = 0.140 +/- 0.001 V
h_max = 0.574 +/- 0.001 V
s = 0.494 +/- 0.001

λ_01 = 0.8748 +/- 0.0009
λ = 3.8 +/- 0.2 cm

x_min_01 = 0.1529 +/- 0.0006
x_min = 0.67 +/- 0.03 cm

βx_min = 1.10 +/- 0.08

ηR_rel = -0.76 +/- 0.02

ξR_rel = 1.23 +/- 0.04

ZR_rel = 1.45 +/- 0.04

P = 
0.11 mW
0.25 mW
0.43 mW
0.63 mW
0.59 mW
0.43 mW
